# CSE5IDP Gartner Project by Agile Avengers

#### Run the below code to install and check YOLO

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.6/107.7 GB disk)


### Pre-requisite functions for gaze detection

In [ ]:
# Functions for gaze detection
import numpy as np

# Function to calculate the midpoint between two points
def calculate_midpoint(point1, point2):
    return [(point1[0] + point2[0]) / 2, (point1[1] + point2[1]) / 2]

def ray_intersection(p1, d1, p2, d2):
    """
    Finds the intersection point of two rays if it exists.
    p1, d1: Starting point and direction vector of the first ray
    p2, d2: Starting point and direction vector of the second ray
    Returns the intersection point if it exists, otherwise None
    """
    # Create vectors from points and directions
    p1, d1 = np.array(p1), np.array(d1)
    p2, d2 = np.array(p2), np.array(d2)

    # Check if they are parallel by cross product
    cross_d1_d2 = np.cross(d1, d2)
    if np.isclose(cross_d1_d2, 0):  # Parallel or identical direction, no intersection
        return None

    # Solving the parametric form of the lines
    t = np.cross(p2 - p1, d2) / cross_d1_d2
    u = np.cross(p2 - p1, d1) / cross_d1_d2

    # Calculate the intersection point if t and u are positive (in forward direction)
    if t > 0 and u > 0:
        intersection_point = p1 + t * d1
        return intersection_point
    else:
        return None

def calculate_inner_angle(d1, d2):
    """
    Calculate the inner angle between two direction vectors
    d1, d2: Direction vectors of the rays
    Returns the inner angle in degrees
    """
    # Normalize vectors
    d1 = d1 / np.linalg.norm(d1)
    d2 = d2 / np.linalg.norm(d2)

    # Calculate angle using dot product, ensuring the inner angle
    dot_product = np.dot(d1, d2)
    angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    return np.degrees(angle)

# Function to calculate the midpoint or use a single eye coordinate if only one is available
def get_eye_point(eye1, eye2):
    if np.all(eye1) and np.all(eye2):  # Check if both eye coordinates are available
        return [(eye1[0] + eye2[0]) / 2, (eye1[1] + eye2[1]) / 2]  # Midpoint of both eyes
    elif np.all(eye1):  # Use eye1 if only eye1 is available
        return eye1
    elif np.all(eye2):  # Use eye2 if only eye2 is available
        return eye2
    return None  # If both are missing, return None

def check_if_in_group(person1, person2, angle_threshold=90):
    """
    Determines if two rays form a group based on their intersection and angle
    ray1_start, ray1_point: Start and direction point for ray 1
    ray2_start, ray2_point: Start and direction point for ray 2
    angle_threshold: Threshold angle to consider the rays as a group
    """
    # Extract nose and eye coordinates for both people
    nose1, eye1a, eye1b = person1[0], person1[1], person1[2]
    nose2, eye2a, eye2b = person2[0], person2[1], person2[2]

    # Check if nose coordinates are available for both
    if not (np.all(nose1) and np.all(nose2)):
        print("Nose coordinate missing; cannot determine direction.")
        return None

    # Get eye points or midpoints for both people
    eye_point1 = get_eye_point(eye1a, eye1b)
    eye_point2 = get_eye_point(eye2a, eye2b)

    # Check if eye points were successfully obtained
    if eye_point1 is None or eye_point2 is None:
        print("Eye coordinates missing; cannot determine direction.")
        return None


    # Define direction vectors
    d1 = np.array(nose1) - np.array(eye_point1)
    d2 = np.array(nose2) - np.array(eye_point2)

    # Check for intersection
    intersection_point = ray_intersection(eye_point1, d1, eye_point2, d2)
    if intersection_point is None:
        print("Rays do not intersect; not in a group.")
        return False

    # Calculate the inner angle
    inner_angle = calculate_inner_angle(d1, d2)
    print(f"Intersection point: {intersection_point}")
    print(f"Inner angle between rays: {inner_angle} degrees")

    # Check if the inner angle is below the threshold
    if inner_angle < angle_threshold:
        print("The rays are in a group based on angle threshold.")
        return True
    else:
        print("The rays are not in a group based on angle threshold.")
        return False

#### After running the above code once you can run the below code as many times you want using different or same pictures. Just make sure to change the image name and tune the hyper parameters accordingly.

In [ ]:
# Main source code

import cv2
import numpy as np
from ultralytics import YOLO
from scipy.spatial import distance
import networkx as nx  # For graph and connected component analysis

# Initialize YOLO model (replace 'yolov8n.pt' with the desired model)
model = YOLO('yolov8n.pt')
model_pose = YOLO('yolov8s-pose.pt')

# Set the image path (replace 'test/9.jpeg' with your actual image)
image_path = 'test/5.jpg'

# Perform detection on the image, only detecting class 0 (person)
results = model.predict(source=image_path, conf=0.4, classes=0, save=True)

# Perform pose detection for keypoints
results_pose = model_pose(image_path, save = True)

# Load the image for visualization
image = cv2.imread(image_path)

# Initialize lists to store bounding box centers, coordinates, and areas
bbox_centers = []
bbox_coords = []
bbox_areas = []
first_three_keypoints = []


# Iterate over each detection result
for result in results:
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding box coordinates
    confidences = result.boxes.conf.cpu().numpy()  # Confidence scores
    class_ids = result.boxes.cls.cpu().numpy()  # Class ids (people in this case)

    # Draw bounding boxes around detected people
    for i, box in enumerate(boxes):
        x_min, y_min, x_max, y_max = box.astype(int)
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green bounding box for person
        print(f"Person detected: Bounding Box Coordinates: {box}, Confidence: {confidences[i]}, Class ID: {class_ids[i]}")

        # Compute the center and area of the bounding box
        center_x = (x_min + x_max) // 2
        center_y = (y_min + y_max) // 2
        area = (x_max - x_min) * (y_max - y_min)

        bbox_centers.append([center_x, center_y])
        bbox_coords.append([x_min, y_min, x_max, y_max])
        bbox_areas.append(area)  # Keep track of bounding box area

# Extract first 3 keypoints for each person and plot them
for result in results_pose:
    if hasattr(result, 'keypoints') and result.keypoints is not None:
        keypoints = result.keypoints.data.cpu().numpy()
        # Get the first 3 keypoints (x, y only) for each person and plot
        for person_keypoints in keypoints:
            person_first_three = []
            for idx, (x, y, _) in enumerate(person_keypoints[:3]):  # Only use the first three keypoints
                cv2.circle(image, (int(x), int(y)), 3, (0, 0, 255), -1)  # Red dot for keypoints
                person_first_three.append([x, y])
            first_three_keypoints.append(person_first_three)

# Convert bbox_centers to a numpy array
bbox_centers = np.array(bbox_centers)
first_three_keypoints = np.array(first_three_keypoints)


# Parameters to tune
min_distance = 50      # Minimum distance between people to be considered in the same group
max_distance = 200     # Maximum distance threshold for grouping
area_threshold_ratio = 0.5  # Bounding box area ratio threshold for distance-based grouping

# Step 1: Graph Construction
# Create a graph where each node is a person, and edges exist between people within the distance threshold
G = nx.Graph()
for i in range(len(bbox_centers)):
    G.add_node(i)

# Add edges between people who are close enough (within social distance constraints) and similar in size
for i in range(len(bbox_centers)):
    for j in range(i + 1, len(bbox_centers)):
        # Compute centroid distance
        dist = distance.euclidean(bbox_centers[i], bbox_centers[j])

        # Calculate area ratio
        area_ratio = min(bbox_areas[i], bbox_areas[j]) / max(bbox_areas[i], bbox_areas[j])

        # Check if both distance and area ratio meet the thresholds
        if min_distance <= dist <= max_distance and area_ratio >= area_threshold_ratio:
            result = check_if_in_group(first_three_keypoints[i], first_three_keypoints[j], angle_threshold=90)
            if result is not False:  # Add edge if result is not explicitly False
                G.add_edge(i, j)


# Step 2: Find Connected Components (Groups)
connected_components = list(nx.connected_components(G))

# Step 3: Filter out single-person groups
group_components = [group for group in connected_components if len(group) > 1]

# Step 4: Visualize groups (Draw bounding boxes around groups)
for group in group_components:
    group_indices = list(group)

    # Get bounding box coordinates for all people in the group
    group_x_min = min(bbox_coords[idx][0] for idx in group_indices)
    group_y_min = min(bbox_coords[idx][1] for idx in group_indices)
    group_x_max = max(bbox_coords[idx][2] for idx in group_indices)
    group_y_max = max(bbox_coords[idx][3] for idx in group_indices)

    # Draw a larger bounding box around the group
    cv2.rectangle(image, (group_x_min, group_y_min), (group_x_max, group_y_max), (255, 0, 0), 3)  # Blue bounding box for group

# Density-based methods: Create a density heatmap based on person locations
def create_density_heatmap(bbox_centers, image_shape, sigma=15):
    heatmap = np.zeros((image_shape[0], image_shape[1]), dtype=np.float32)

    for center in bbox_centers:
        x, y = int(center[0]), int(center[1])
        heatmap[y, x] += 1  # Increment heatmap at each person's location

    heatmap = cv2.GaussianBlur(heatmap, (0, 0), sigma)
    heatmap = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    heatmap_colored = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    return heatmap_colored

# Generate heatmap
image_shape = image.shape[:2]
heatmap = create_density_heatmap(bbox_centers, image_shape)

# Save and display the final output with both individual and group bounding boxes
cv2.imwrite('group_detection_output.jpg', image)
cv2.imwrite('density_heatmap.jpg', heatmap)

# Draw all detected pose points on the final output image
for result in results_pose:
    if hasattr(result, 'keypoints') and result.keypoints is not None:
        keypoints = result.keypoints.data.cpu().numpy()

        for person_keypoints in keypoints:
            # Loop through all keypoints for this person and draw them on the image
            for (x, y, _) in person_keypoints:
                x, y = int(x), int(y)
                cv2.circle(image, (x, y), 5, (0, 0, 255), -1)  # Red dots for keypoints with increased size

# Save and display the final output with bounding boxes and keypoints
cv2.imwrite('group_detection_output_with_keypoints.jpg', image)


image 1/1 /content/test/1.jpeg: 288x640 8 persons, 117.0ms
Speed: 2.0ms preprocess, 117.0ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs/detect/predict4


Person detected: Bounding Box Coordinates: [     854.24      131.08      992.27       472.2], Confidence: 0.8006943464279175, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     495.21      144.98      624.65       454.4], Confidence: 0.7956050038337708, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     745.91      126.59       885.6      484.88], Confidence: 0.743084728717804, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     346.73      163.74      396.99      360.07], Confidence: 0.7022331357002258, Class ID: 0.0
Person detected: Bounding Box Coordinates: [      211.4      114.76      357.62      403.85], Confidence: 0.5746599435806274, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     184.18      194.09      320.98      430.19], Confidence: 0.5475128889083862, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     213.11      115.09      356.87      296.37], Confidence: 0.4261578619480133, Class ID: 0.0
Person detected: Boun

True